In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools as it
from collections import defaultdict
import seaborn as sns

In [2]:
path_common_source_trans = 'train_transaction.csv'
path_common_source_idn = 'train_identity.csv'

In [3]:
train_trs = pd.read_csv(path_common_source_trans)
train_idn = pd.read_csv(path_common_source_idn )

In [4]:
data_merged = pd.merge(train_trs, train_idn, left_on='TransactionID', right_on='TransactionID', how='left')

In [5]:
def get_share_of_NaN(df):
    result = pd.DataFrame(columns=['Name', 'Number_of_NaN', 'Share_of_NaN'])
    colcount = df.count()
    length = len(df)
    for col_name in colcount.keys():
        result.loc[len(result)] = [col_name, length-colcount[col_name], (length-colcount[col_name])/length]
    return result

def remove_columns_with_many_NaN(df, max_nan_rate):
    '''
    Параметры:
    df - DataFrame
    max_nan_rate - максимальная допустимая доля NaN в колонках датафрейма
    Функция возвращает:
    1) новый датафрейм, в котором удалены колонки, в которых доля NaN болше, чем max_nan_rate
    2) список удалённых колонок
    '''
    df_copy = df.copy()
    removed_columns = []
    nan_stat = get_share_of_NaN(df)
    for i in range(len(nan_stat)):
        column = nan_stat.loc[i]
        if(column['Share_of_NaN'] > max_nan_rate):
            removed_columns.append(column['Name'])
            
    df_copy.drop(columns=removed_columns, inplace=True)
    return df_copy, removed_columns

def remove_columns_with_big_correlation(df, max_corr):
    '''
    Параметры:
    df - DataFrame
    max_corr - максимальная допустимая корреляция между колонками
    Функция возвращает:
    1) новый датафрейм, в котором удалены колонки, в которых корреляция болше, чем max_corr
    2) множество удалённых колонок
    '''
    df_copy = df.copy()
    removed_columns = set()
    corrs = df.corr()
    cols = corrs.columns
    for i in range(len(cols)):
        col_name_1 = cols[i]
        if col_name_1 in {'TransactionID', 'isFraud', 'TransactionDT'} or col_name_1 in removed_columns:
            continue
        
        for j in range(i+1, len(cols)):
            col_name_2 = cols[j]
            if abs(corrs[col_name_1][col_name_2]) > max_corr:
                removed_columns.add(col_name_2)

    df_copy.drop(columns=removed_columns, inplace=True)
    return df_copy, removed_columns

In [6]:
%%time
data_merged, removed_nan_cols = remove_columns_with_many_NaN(data_merged, 0.85)
data_merged_rm_cols, removed_corr_cols = remove_columns_with_big_correlation(data_merged, 0.9)

CPU times: total: 2min 25s
Wall time: 2min 26s


In [7]:
cat_features_mask = (data_merged_rm_cols.dtypes == "object").values
val_features_mask = (data_merged_rm_cols.dtypes != "object").values
for i in range(len(data_merged_rm_cols.columns)):
    if data_merged_rm_cols.iloc[:,i].name in ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2']:
        cat_features_mask[i] = True
        val_features_mask[i] = False

cat_cols = data_merged_rm_cols[data_merged_rm_cols.columns[cat_features_mask]]
var_cols = data_merged_rm_cols[data_merged_rm_cols.columns[~cat_features_mask]]
# print(cat_cols)

In [20]:
# remove all nones
def proc_cat_col(df, cat_features_mask):
    cat_cols = df.columns[cat_features_mask]
    #print(cat_cols)
    for col in cat_cols:
        dc = df.loc[:,col]
#         dc = dc.fillna(dc.mode()[0])
        dc.fillna(dc.mode()[0], inplace=True)


def proc_val_col(df, cat_features_mask, sample_size):
    val_cols = df.columns[~cat_features_mask]
    for col in val_cols:
        dc = df.loc[:,col]
#         dc = dc.fillna(dc.sample(n=sample_size).dropna().mode()[0])
        dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)

In [9]:
data_merged.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V321,id_01,id_02,id_05,id_06,id_11,id_13,id_17,id_19,id_20
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,581607.000000,588975.000000,586281.000000,524834.000000,524834.000000,...,590528.000000,144233.000000,140872.000000,136865.000000,136865.000000,140978.000000,127320.000000,139369.000000,139318.000000,139261.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,362.555488,153.194925,199.278897,290.733794,86.800630,...,28.326584,-10.170502,174716.584708,1.615585,-6.698710,99.745325,48.053071,189.451377,353.128174,403.882666
std,1.704744e+05,0.183755,4.617224e+06,239.162522,4901.170153,157.793246,11.336444,41.244453,101.741072,2.690623,...,382.053171,14.347949,159651.816856,5.249856,16.491104,1.127602,11.774858,30.375360,141.095343,152.160327
min,2.987000e+06,0.000000,8.640000e+04,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,0.000000,-100.000000,1.000000,-72.000000,-100.000000,90.000000,10.000000,100.000000,100.000000,100.000000
25%,3.134635e+06,0.000000,3.027058e+06,43.321000,6019.000000,214.000000,150.000000,166.000000,204.000000,87.000000,...,0.000000,-10.000000,67992.000000,0.000000,-6.000000,100.000000,49.000000,166.000000,266.000000,256.000000
50%,3.282270e+06,0.000000,7.306528e+06,68.769000,9678.000000,361.000000,150.000000,226.000000,299.000000,87.000000,...,0.000000,-5.000000,125800.500000,0.000000,0.000000,100.000000,52.000000,166.000000,341.000000,472.000000
75%,3.429904e+06,0.000000,1.124662e+07,125.000000,14184.000000,512.000000,150.000000,226.000000,330.000000,87.000000,...,0.000000,-5.000000,228749.000000,1.000000,0.000000,100.000000,52.000000,225.000000,427.000000,533.000000
max,3.577539e+06,1.000000,1.581113e+07,31937.391000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,104060.000000,0.000000,999595.000000,52.000000,0.000000,100.000000,64.000000,229.000000,671.000000,661.000000


In [21]:
%%time
data_f0 = data_merged_rm_cols[data_merged_rm_cols['isFraud'] == 0]
data_f1 = data_merged_rm_cols[data_merged_rm_cols['isFraud'] == 1]
proc_cat_col(data_f0, cat_features_mask)
proc_val_col(data_f0, cat_features_mask,10000)
proc_cat_col(data_f1, cat_features_mask)
proc_val_col(data_f1, cat_features_mask,10000)
clear_and_nan = pd.concat([data_f0, data_f1])
clear_and_nan.describe()

C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.mode()[0], inplac

C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.mode()[0], inplac

C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

...

C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dc.fillna(dc.sample(n=sample_size).dropna().mode()[0], inplace=True)
C:\Users\Art\AppData\Local\Temp\ipykernel_12316\3750313232.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V320,id_01,id_02,id_05,id_06,id_11,id_13,id_17,id_19,id_20
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,...,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,362.087335,153.186458,199.471611,290.408235,86.822813,...,42.072278,-6.262839,68591.496397,0.374432,-1.552510,99.939202,51.149045,172.495408,286.554955,482.682895
std,1.704744e+05,0.183755,4.617224e+06,239.162522,4901.170153,156.750049,11.322604,41.157610,96.461039,2.537300,...,473.494534,7.430643,98125.796874,2.617689,8.427247,0.561537,5.703210,19.027392,77.877999,85.883911
min,2.987000e+06,0.000000,8.640000e+04,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,0.000000,-100.000000,1.000000,-72.000000,-100.000000,90.000000,10.000000,100.000000,100.000000,100.000000
25%,3.134635e+06,0.000000,3.027058e+06,43.321000,6019.000000,215.000000,150.000000,166.000000,204.000000,87.000000,...,0.000000,-5.000000,36097.000000,0.000000,0.000000,100.000000,52.000000,166.000000,266.000000,507.000000
50%,3.282270e+06,0.000000,7.306528e+06,68.769000,9678.000000,360.000000,150.000000,226.000000,299.000000,87.000000,...,0.000000,-5.000000,36097.000000,0.000000,0.000000,100.000000,52.000000,166.000000,266.000000,507.000000
75%,3.429904e+06,0.000000,1.124662e+07,125.000000,14184.000000,512.000000,150.000000,226.000000,327.000000,87.000000,...,0.000000,-5.000000,36097.000000,0.000000,0.000000,100.000000,52.000000,166.000000,266.000000,507.000000
max,3.577539e+06,1.000000,1.581113e+07,31937.391000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,104060.000000,0.000000,999595.000000,52.000000,0.000000,100.000000,64.000000,229.000000,671.000000,661.000000


In [22]:
clear_and_nan.describe(include='object')

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,...,id_16,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
count,590540,590540,590540,590540,590540,590540,590540,590540,590540,590540,...,590540,590540,590540,590540,590540,590540,590540,590540,590540,590540
unique,5,4,4,59,60,2,2,2,3,2,...,2,2,2,130,2,2,2,2,2,1786
top,W,visa,debit,gmail.com,gmail.com,T,T,T,M0,F,...,NotFound,Found,Found,chrome 63.0,T,F,T,F,desktop,Windows
freq,439670,386344,441509,322811,510396,590515,556568,522831,477849,469879,...,513705,525794,524488,472258,517928,583621,560007,523477,525443,519596


In [19]:
clear_and_nan['id_15'].unique()

array(['Found', 'New', 'Unknown'], dtype=object)

In [13]:
clear_and_nan.sort_values(by=['TransactionID'], inplace=True)

In [14]:
clear_and_nan.to_csv('clear_and_nan.csv', index=False)

In [18]:
clear_and_nan['DeviceType'].notna().sum()

590540